In [9]:
# Basic imports
import torch
import torchvision
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.functional as F
import subprocess
from matplotlib import pyplot as plt
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import ConcatDataset, DataLoader
import os

In [ ]:
class MLPShallow(nn.Module):
    def __init__(self):
        super(MLPShallow, self).__init__()
        self.input_layer = nn.Linear(28 * 28, 512)
        self.hidden_layer_1 = nn.Linear(512, 256)
        self.hidden_layer_2 = nn.Linear(256, 128)
        self.hidden_layer_3 = nn.Linear(128, 64)
        self.output_layer = nn.Linear(64, 10)

        self.dropout1 = nn.Dropout(0.2)
        self.dropout2 = nn.Dropout(0.2)

    def forward(self, x):
        x = x.view(-1, 28 * 28)

        x = F.relu(self.input_layer(x))
        x = self.dropout1(x)

        x = F.relu(self.hidden_layer_1(x))
        x = self.dropout1(x)

        x = F.relu(self.hidden_layer_2(x))
        x = self.dropout2(x)

        x = F.relu(self.hidden_layer_3(x))
        x = self.dropout2(x)

        x = self.output_layer(x)
        return x

model = MLPShallow()

In [ ]:
def to_binary(image):
    threshold = 0.5
    return (image > 0).float()

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
    transforms.Lambda(to_binary)
])

training_data_mnist = torchvision.datasets.MNIST(root="../data", train=True, transform=transform, download=True)
test_data_mnist = torchvision.datasets.MNIST(root="../data", train=False, transform=transform, download=True)

training_data_emnist = torchvision.datasets.EMNIST(root="../data", split="digits", train=True, transform=transform, download=True)
test_data_emnist = torchvision.datasets.EMNIST(root="../data", split="digits", train=False, transform=transform, download=True)

full_dataset_train = ConcatDataset([training_data_mnist, training_data_emnist])
full_dataset_test = ConcatDataset([test_data_mnist, test_data_emnist])

batch_size = 64
data_loader_train = DataLoader(full_dataset_train, batch_size=batch_size, shuffle=True)
data_loader_test = DataLoader(full_dataset_test, batch_size=batch_size, shuffle=True)

In [ ]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

In [ ]:
Epochs = 3
for epoch in range(Epochs):
  training_loss = 0.0
  correct = 0
  total = 0
  for i, data in enumerate(data_loader_train, 0):
    inputs, labels = data
    inputs = inputs.view(inputs.shape[0], -1)

    optimizer.zero_grad()

    outputs = model(inputs)
    loss = loss_function(outputs, labels)
    loss.backward()
    optimizer.step()

    training_loss += loss.item()
    _, predicted = outputs.max(1)
    total += labels.size(0)
    correct += predicted.eq(labels).sum().item()
    avg_loss = training_loss / (i + 1)
    avg_acc = 100. * correct / total
  print(f'Training Loss: {avg_loss:.3f} | Training acc: {avg_acc:.3f}', 'for epoch: ', epoch)

Training Loss: 0.481 | Training acc: 87.699 for epoch:  0
Training Loss: 0.266 | Training acc: 93.440 for epoch:  1
Training Loss: 0.225 | Training acc: 94.519 for epoch:  2


In [14]:
torch.save(model.state_dict(), 'cnn_6.pth')
print("Model saved to 'cnn.pth'")

Model saved to 'cnn.pth'


In [ ]:
model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for i, (image, label) in enumerate(data_loader_test):
      image = image.view(image.shape[0], -1)
      output = model(image)
      test_loss += F.nll_loss(output, label, reduction='sum').item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(label.data.view_as(pred)).sum()
    test_loss /= len(data_loader_test.dataset)
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(data_loader_test.dataset), 100. * correct / len(data_loader_test.dataset)))


Test set: Avg. loss: -7.0817, Accuracy: 48826/50000 (98%)

